In [20]:
# Parameters for the calculation of the reaction rates
SIGV_PARAMS = [
    # [a1, a2, a3, a4, a5, a6, a7]
    [-38.5922, -0.00630249, 1.25159, -8.86637, 1.78419, -0.198181, 1.92028], # Reaction 2
    [-46.8616, -0.00295519, -3.23527, 1.42088, -0.126344, 0.00974799, -0.178045], # Reaction 3
    [-42.1159, 0.0101979, -6.1653, 10.9227, -0.654152, 0.0352096, -3.41782], # Reaction 4
    [-43.1056, 0.0150095, -6.85042, 12.3633, -0.638696, 0.0261701, -4.16095], # Reaction 5
    [-34.1675, -0.523, 4.63833e-10, -1.67345e-9, 1.85876e-10, -1.70346e-11, -0.666667], # Reaction 6
    [-31.7974, -0.0137498, -5.56986, -14.1977, 2.27582, -0.19545, 2.92443], # Reaction 7
    [-34.1075, -0.000673532, 0.0793345, -0.532712, 0.13522, -0.00748114, 0.104904], # Reaction 8
    [-34.8617, -1.72, -4.68818e-9, 1.21488e-8, -1.10092e-9, 8.94955e-11, -0.5], # Reaction 9
    [-38.9414, -0.000245919, -12.7758, -0.489479, -0.0881085, 0.00557341, -0.582926], # Reaction 10
    [-32.5903, -0.0032998, -7.83923, -2.1531, 0.133483, -0.00732414, 0.20178], # Reaction 11
    [-32.3946, 9.06317e-7, -2.04247e-9, -1.167, -4.39104e-10, 3.41812e-11, -1.98589e-9] # Reaction 12
]

In [21]:
from mendeleev import element
from mendeleev import isotope
import scipy as sci
import scipy.constants as constant
import numpy as np

In [24]:
#Konstanten
#Temperatur in GK
#Masse in MeV
kB=0.0861734 #in MeV/GK
kBG=0.0861734 *10**(-3) #in GeV/GK
hsc=1.97327 * 10**(-11) #in MeV*cm
hscG=hsc*10**(-3) #in GeV*cm
c=sci.constants.c #in cm/s
muc2= 931.494 # uc2 in MeV
Joule_to_GeV = 1/(sci.constants.e*10**6)
lambda_beta=0.00113443 #in s^-1
nu=6.2 * 10**(-10)

## Hilfsmittel und Konstanten

In [25]:
class Particle:
    def __init__(self,mass,Z,A,j):
        self.mass = mass * 1.66053906660*10**(-27)
        self.Z = Z
        self.A = A
        self.spin = Spin_to_float(j)

In [26]:
def contains_slash(string):
    if "/" in string:
        return True
    else:
        return False

def Spin_to_float(j):
    if (contains_slash(j)==True):
        # Split the string into two parts using the forward slash
        parts = j.split("/")
        # Convert the numerator and denominator to floats
        numerator = float(parts[0])
        denominator = float(parts[1])
        # Divide the numerator by the denominator
        result = numerator / denominator
        return result
    
    else:
        return float(j)

def kron_delta(part1,part2):
    if (part1 == part2):
        return 1
    else:
        return 0

    

In [27]:
n=Particle(1.00866491595,0,1,'1/2')
H1= Particle(isotope(1, 1).mass,1,1,isotope(1, 1).spin) 
H2= Particle(isotope(1, 2).mass,1,2,isotope(1, 2).spin)
H3=Particle(isotope(1, 3).mass,1, 3,isotope(1, 3).spin)
He3=Particle(isotope(2, 3).mass,2, 3,isotope(2, 3).spin)
He4=Particle(isotope(2, 4).mass,2, 4,isotope(2, 4).spin)
Li7=Particle(isotope(3, 7).mass,3, 7,isotope(1, 2).spin)
Be7=Particle(isotope(4, 7).mass,4, 7,isotope(4,7).spin)

#Just in case it becomes more comfortable to write them like this:
p=H1 
d=H2
t=H3


## Nicht-Differentialgleichungen

In [37]:

def n(T): #T muss in GK eingegeben werden
    return nu*(2*sci.special.zeta(3)*(kB*T)**3)/(np.pi *hsc)**3



In [38]:
def ov_T(T,r_number):
    a1, a2,a3,a4,a5,a6,a7=SIGV_PARAMS[reaction_number-2]#Da 1. Reaktion das Beta Zerfall ist und das index mit 0 anfängt
    return np.exp(a1 + a2/T + a3*T**(-1/3) + a4*T**(1/3) + a5*T +a6*T**(5/3) + a7*np.log(T))

def lambda_gamma(T,part1,part2,part3,reaction_number):
    Term1=(muc2*kB*T/(2*(sci.constants.pi)**2 * hsc**2))**(3/2)
    Term2=(2*part1.spin +1)*(2*part2.spin +1)/(2*part3.spin +1)
    Term3=(part1.A*part2.A/part3.A)**(3/2)
    Q=(part1.mass + part2.mass - part3.mass)*muc2 #mc^2 in Joule   
    Term4=(Q*10**-3)/(kB*T) #Q*10**-3 sodass Q in GeV ist
    Term5=ov_T(T,reaction_number)/(1+k_delta_part(part1,part2))
    tog=Term1*Term2*Term3*Term4*Term5
    return tog



In [39]:
def ov_cd(T,part1,part2,part3,part4,r_number):
    Term1=(1+k_delta_part(part3,part4))/(1+k_delta_part(part1,part2))
    Term2=(2*part1.spin +1)*(2*part2.spin +1)/((2*part3.spin +1)*(2*part4.spin +1))
    Term3=(part1.A*part2.A/(part3.A*part4.A))**(3/2)
    Q=u_to_kg*(part1.mass + part2.mass - part3.mass - part4.mass)*c**2 #mc^2 in Joule 
    Term4=(Q*Joule_to_GeV)/(kB*T)
    tog=Term1*Term2*Term3*Term4* ov_T(T,r_number)
    return tog

In [40]:
def T(t):
    return 14.0/np.sqrt(t)

## Differentialgleichung

In [30]:
def dt_to_dT(T):
    return -T**3 /392,0 

In [ ]:
def drv_gammapart(T,N_liste,part1,part2,part3,r_number,n): #Das ist für dyDT
     
    Ya = N_liste[0+(r_number-1)*4]
    Yb = N_liste[1+(r_number-1)*4]
    Yc = N_liste[2+(r_number-1)*4]
    dYa=-Ya*Yb*n(T)*ov_T(T,r_number)+(1+kron_delta(part1,part2))*lambda_gamma(T,part1,part2,part3,reaction_number)*Yc
    dYb=-Ya*Yb*n(T)*ov_T(T,r_number)+(1+kron_delta(part1,part2))*lambda_gamma(T,part1,part2,part3,reaction_number)*Yc
    dYc=Ya*Yb*n(T)*ov_T(T,r_number)/(1+kron_delta(part1,part2))-lambda_gamma(T,part1,part2,part3,reaction_number)*Yc
    return[dYa,dYb,dYc]





In [ ]:
def drv_forthpart(T,N_liste,part1,part2,part3,part4,r_number,n):
         
    Ya = N_liste[0+(r_number-1)*4]
    Yb = N_liste[1+(r_number-1)*4]
    Yc = N_liste[2+(r_number-1)*4]
    Yd = N_liste[3+(r_number-1)*4]
    Term1 = -Ya * Yb * n * ov_T(T,r_number)
    Term2 = ((1 + k_delta_part(part3, part4)) / (1 + k_delta_part(part1, part2)))**(-1) *  n(T) * Yc * Yd * ov_cd(part1, part2, part3, part4)
    Term3 = (1 + k_delta_part(part3, part4)) / (1 + k_delta_part(part1, part2)) * Ya * Yb * n(T) * ov_T(T,r_number)
    Term4 = -ov_cd(part1, part2, part3, part4) *  n(T) * Yc * Yd
    
    dYa= Term1+Term2
    dYb=Term1+Term2
    dYc=Term3+Term4
    dYd=Term3+Term4
    
    return[dYa,dYb,dYc,dYd]